In [54]:
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


# Tag the Tweet!

Tagging a Tweet based on its context! Using Recurrent Neural Networks, NGrams.

# Problem Statement

        Tag the Tweet is an Application built on python, which understands all the tweets of the trending hashtags
        and predicts a hashtag for the given tweets.

In [55]:
import numpy as np
import twitter
import sys
import re, datetime, pandas as pd
import tweepy as tw
#from ResultMiner import TweetMiner

# Step - 01

We need a lot of data regarding the Tweets! Hence I am using `tweepy` to authenticate my Twitter Keys and use the API to retrieve the required data.

In [66]:
MAX_HASHTAGS_COUNT = 3
MIN_HASHTAGS_COUNT = 2
US_WOE_ID = 23424977
INDIA_WOE_ID = 23424848
RUSSIA_WOE_ID = 23424936
TWEET_LIMIT = 125

twitter_details = {
    'consumer_key'        : "pNPFulq7iU1AwsH8WKXcM3VhZ",
    'consumer_secret'     : "k6weUOaZxYSbBBLCBB5DsshL8iUDWIOLY5WlHK555kHRWOinuQ",
    'access_token_key'    : "782632596-Zx3pED7JikyG7dBx1a6oIqyQh60qeSMUXHAqjfhg",
    'access_token_secret' : "DoqjnFjyJIqEEsqUVtnyWX2KYyswA9rTCPtJr9GGfyTHL"
}

twitter_api = twitter.Api(consumer_key= twitter_details['consumer_key'],
                  consumer_secret     = twitter_details['consumer_secret'],
                  access_token_key    = twitter_details['access_token_key'],
                  access_token_secret = twitter_details['access_token_secret'])

auth = tw.OAuthHandler(twitter_details['consumer_key'], twitter_details['consumer_secret'])
auth.set_access_token(twitter_details['access_token_key'], twitter_details['access_token_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

In [67]:
class TweetMiner():

    def __init__(self, api, tweet_limit):
        
        self.api = api
        self.tweet_limit = tweet_limit
        
        
    def retrieve_data(self, tag, WOE_ID, page_limit):
        
        cleaned_data = []
        latest_max_id = False
        jk = True
        for i in range(page_limit):
            tweets = tw.Cursor(self.api.search,
                     q=tag+" -filter:retweets",
                     lang="en",
                     max_id = latest_max_id,
                     since=2018-1-1).items(self.tweet_limit)
            
            for tweet in tweets:
                try:
                    data = {
                        'id' : tweet.id,
                        'user_name' : tweet.user.screen_name,
                        'tweet_description' : tweet.text,
                        'retweet_count' : tweet.retweet_count,
                        'created_at' : tweet.created_at,
                        'hashtag' : tag
                    }
                except:
                    data = {
                        'id' : tweet.id,
                        'user_name' : tweet.user.screen_name,
                        'tweet_description' : tweet.text,
                        'retweet_count' : 0,
                        'created_at' : tweet.created_at,
                        'hashtag' : tag
                    }
                latest_max_id = tweet.id
                cleaned_data.append(data)
        
        return cleaned_data

In [68]:
tweet_miner = TweetMiner(api, TWEET_LIMIT)

In [92]:
def get_trending_hashtags(trends):
    trends = trends['trends']
    trending_hashtags = []
    for trend in trends:
        print(str(trend['tweet_volume']) + " " + trend['name'])
        if trend['name'][0] == '#' and trend['tweet_volume'] != None and trend['tweet_volume'] >= 2500:
            trending_hashtags = trend
            break
    return trending_hashtags

In [99]:
hashtags = []
hashtags.append(get_trending_hashtags(api.trends_place(US_WOE_ID)[0]))
hashtags.append(get_trending_hashtags(api.trends_place(INDIA_WOE_ID)[0]))
hashtags.append(get_trending_hashtags(api.trends_place(RUSSIA_WOE_ID)[0]))
data = []

1410082 Happy Easter
12706 #StuffWeShouldCancel
57119 #MUNBHA
None #маска
None #ростовспартак
None С Победой
None #голоскраїни
None #мгчд
132513 #RestoreTheSnyderVerse


# Tweet Retrievel from Trending HashTags
  We are trying to understand three popular hashtags by mining 1500 tweets each..
  Below are the pandas dataframes for each of these. Which we later try to merge back
  as on dataset X

In [100]:
print(hashtags[0])
print("_________________________")
print(hashtags[1])
print("_________________________")
print(hashtags[2])
print("_________________________")


{'name': '#StuffWeShouldCancel', 'url': 'http://twitter.com/search?q=%23StuffWeShouldCancel', 'promoted_content': None, 'query': '%23StuffWeShouldCancel', 'tweet_volume': 12706}
_________________________
{'name': '#MUNBHA', 'url': 'http://twitter.com/search?q=%23MUNBHA', 'promoted_content': None, 'query': '%23MUNBHA', 'tweet_volume': 57119}
_________________________
{'name': '#RestoreTheSnyderVerse', 'url': 'http://twitter.com/search?q=%23RestoreTheSnyderVerse', 'promoted_content': None, 'query': '%23RestoreTheSnyderVerse', 'tweet_volume': 132513}
_________________________


In [106]:
data.append(tweet_miner.retrieve_data(hashtags[0]['name'], US_WOE_ID, 20))
US_df = pd.DataFrame(data[0])

In [107]:
US_df

,id,user_name,tweet_description,retweet_count,created_at,hashtag
0,1378822809124810758,BillHog15360397,#StuffWeShouldCancel Changing our clocks twic...,0,2021-04-04 21:32:29,#StuffWeShouldCancel
1,1378822753042624516,Lunaursaa,#StuffWeShouldCancel \nThe human race,0,2021-04-04 21:32:16,#StuffWeShouldCancel
2,1378822752904286208,RealMitcHilburn,#StuffWeShouldCancel\n\nFinal season of:\n#Gam...,0,2021-04-04 21:32:16,#StuffWeShouldCancel
3,1378822715784753154,pbyrond,@SteveInmanUIC Steve does this fall into the #...,0,2021-04-04 21:32:07,#StuffWeShouldCancel
4,1378822667319529472,JessieBrewer,"#StuffWeShouldCancel Ivanka, Tiffany, Barron, ...",0,2021-04-04 21:31:55,#StuffWeShouldCancel
...,...,...,...,...,...,...
2495,1378769957383589888,kindnessgang,i feel like cancel culture and accountability ...,0,2021-04-04 18:02:28,#StuffWeShouldCancel
2496,1378769941999054855,jordantimes5,Youngins that don’t know a damn thing about mu...,0,2021-04-04 18:02:25,#StuffWeShouldCancel
2497,1378769941235654658,KeelaYoung1,@QasimRashid The Kardashians #StuffWeShouldCancel,0,2021-04-04 18:02:24,#StuffWeShouldCancel
2498,1378769927214092288,terryturner1978,#StuffWeShouldCancel \nPeople that think #Bitc...,0,2021-04-04 18:02:21,#StuffWeShouldCancel


In [108]:
data.append(tweet_miner.retrieve_data(hashtags[1]['name'], INDIA_WOE_ID, 20))
India_df = pd.DataFrame(data[1])

In [109]:
India_df

,id,user_name,tweet_description,retweet_count,created_at,hashtag
0,1378825559862607873,hollybarbie94,"#OleOut will never be a thing for me, he's doi...",0,2021-04-04 21:43:25,#MUNBHA
1,1378825513486049280,AU0481810206,"India won over their ""masters""\n\n#auspol Aust...",0,2021-04-04 21:43:14,#MUNBHA
2,1378825235399647235,RankingTrend,Worldwide Twitter trend ranking \n1.#HappyEast...,0,2021-04-04 21:42:08,#MUNBHA
3,1378825091736334343,DaliaKhashan,3 points with a weak performance. We move on.....,0,2021-04-04 21:41:33,#MUNBHA
4,1378825038061830152,DerrenHoward01,This is very good from @LoganMacLeod8 at Old T...,0,2021-04-04 21:41:21,#MUNBHA
...,...,...,...,...,...,...
2495,1378804383757053956,mojdris,Greenwood 🔥🔥🔥#MUNBHA,0,2021-04-04 20:19:16,#MUNBHA
2496,1378804379558445056,fazalshaikh1,Bissouma in that Brighton Midfield is awesome....,0,2021-04-04 20:19:15,#MUNBHA
2497,1378804378660917255,sadoh_edmond,Mason😣😣❤️....\n\n#MUFC #MUNBHA,0,2021-04-04 20:19:15,#MUNBHA
2498,1378804378296053766,Lamola_Herold,GREEN-wood gives The RED Devils the lead.\n\nF...,0,2021-04-04 20:19:15,#MUNBHA


In [110]:
data.append(tweet_miner.retrieve_data(hashtags[2]['name'], RUSSIA_WOE_ID, 20))
UK_df = pd.DataFrame(data[2])

In [111]:
UK_df

,id,user_name,tweet_description,retweet_count,created_at,hashtag
0,1378830202458017797,castrogacio,@kryptoncaged Don’t like secrets! 😡\n\n#Restor...,0,2021-04-04 22:01:52,#RestoreTheSnyderVerse
1,1378830193628999681,smokingroze,#RestoreTheSnyderVerse for the Green Lantern s...,0,2021-04-04 22:01:50,#RestoreTheSnyderVerse
2,1378830182547660801,phantomx1313,Restore The SnyderVerse because everyone wants...,0,2021-04-04 22:01:47,#RestoreTheSnyderVerse
3,1378830178764390402,JamesKaltenbac3,@capecastshow Hell yea😎🤘🤘#ZackSnydersJusticeLe...,0,2021-04-04 22:01:46,#RestoreTheSnyderVerse
4,1378830150792409094,Nabilahlegolas,@pingpongflix We need zack to continue his arc...,0,2021-04-04 22:01:40,#RestoreTheSnyderVerse
...,...,...,...,...,...,...
2495,1378794119003258882,SkyDracoBlue,#ContinueTheMonsterverse\n#RestoreTheSnyderVer...,0,2021-04-04 19:38:29,#RestoreTheSnyderVerse
2496,1378794104784625664,DavidStegen01,For Slade!🔥 #RestoreTheSnyderVerse \n\n@hbomax...,0,2021-04-04 19:38:25,#RestoreTheSnyderVerse
2497,1378794100854505473,Andrea24976180,For #Darkseid @Ray__Porter in ZSJL Part 2 #Res...,5,2021-04-04 19:38:25,#RestoreTheSnyderVerse
2498,1378794097360695296,Mr_Jim_G,@inversedotcom @AllThingsSnyder That's why we ...,0,2021-04-04 19:38:24,#RestoreTheSnyderVerse


# Ngrams

Ngrams helps us in understanding intresting things about the corpus.

## For example 
We can know the similarity of each tweet under a specific tag using the Ngrams. 
For this purpose we shall:

1) 

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from collections import Counter

vect = tfidf(ngram_range=(2,5), stop_words='english')

summaries = "".join(US_df['tweet_description'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('stuffweshouldcancel stuffweshouldcancel', 377),
 ('stuffweshouldcancel https', 153),
 ('cancel culture', 123),
 ('stuffweshouldcancel cancel', 111),
 ('stuffweshouldcancel cancel culture', 72),
 ('culture stuffweshouldcancel', 53),
 ('stuffweshouldcancel people', 51),
 ('cancel culture stuffweshouldcancel', 39),
 ('stuffweshouldcancel stuffweshouldcancel cancel', 32),
 ('stuffweshouldcancel twitter', 24),
 ('stuffweshouldcancel cancel culture stuffweshouldcancel', 24),
 ('republican party', 22),
 ('stuffweshouldcancel student', 22),
 ('student debt', 20),
 ('stuffweshouldcancel stuffweshouldcancel cancel culture', 19),
 ('loan debt', 16),
 ('twitter stuffweshouldcancel', 16),
 ('student loan', 16),
 ('culture stuffweshouldcancel stuffweshouldcancel', 16),
 ('stuffweshouldcancel canceling', 15)]

In [115]:
vect = tfidf(ngram_range=(2,5), stop_words='english')

summaries = "".join(India_df['tweet_description'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('munbha https', 355),
 ('manchester united', 191),
 ('munbha mufc', 127),
 ('mufc munbha', 109),
 ('second half', 77),
 ('united brighton', 73),
 ('man united', 67),
 ('mufc munbha https', 65),
 ('man utd', 60),
 ('united fans', 49),
 ('mason greenwood', 48),
 ('manchester united brighton', 46),
 ('mufc https', 42),
 ('premier league', 38),
 ('rashford greenwood', 37),
 ('manutd munbha', 35),
 ('munbha munbha', 35),
 ('old trafford', 34),
 ('munbhamanchester united', 33),
 ('munbha mufc https', 33)]

In [116]:
vect = tfidf(ngram_range=(2,5), stop_words='english')

summaries = "".join(UK_df['tweet_description'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('restorethesnyderverse https', 760),
 ('zacksnydersjusticeleague restorethesnyderverse', 92),
 ('restorethesnyderverse zacksnydersjusticeleague', 81),
 ('restorethesnyderverse restorethesnyderverse', 67),
 ('zacksnydersjusticeleague restorethesnyderverse https', 64),
 ('zacksnydersjusticeleague https', 63),
 ('justice league', 63),
 ('restorethesnyderverse zacksnydersjusticeleague https', 44),
 ('zack snyder', 42),
 ('att jasonkilar', 39),
 ('hbomax restorethesnyderverse', 38),
 ('restorethesnyderverse hbomax', 38),
 ('restorethesnyderverse att', 32),
 ('watching zacksnydersjusticeleague', 31),
 ('snyder justice', 30),
 ('snyder justice league', 30),
 ('att hbomax', 28),
 ('time restorethesnyderverse', 27),
 ('hbomax att', 24),
 ('hbomax warnermedia', 23)]

# Prepare the Data

Now, we need to concat the data we collected from three of these hashtags.

1) Concate the data to form a big data set, using which we build a learning model.
2) Clean and shuffle this data.
3) divide this data into training and test sets.

In [117]:
tweets_data = pd.concat([US_df, India_df, UK_df], axis =0)
tweets_data.to_csv('hastags_dataframes.csv')